In [12]:
def thresh(img, val, percent = 1, max_val = .1):
    
        #convert to positive values
        img_filt = img[~np.isnan(img)] *-1
        
        #define upper maximum
        img_filt = img_filt[img_filt<max_val]
        
        #total cell count
        tot_cell = len(img_filt)
        
        #threshold
        threshold_val = val*percent
        
        #number above thresh
        counts = len(img_filt[img_filt>=threshold_val])/tot_cell
        return counts

In [22]:
# This function takes in a function and various constant and returns a np_array
# func - Function to be run on contractile data
# val_arr - Contractile Data
# peak_arr - peak counts array
# data_len - length of data
# *func_args - additional arguments not included for above function
# RETURN - np_array

def funcToPlot(func, val_arr, peak_arr, data_len, *func_args):
    #create empty dict to be populated
    e_dict = dict()
    
    # loop through all contractile data sets
    for i in range(data_len):
        y = val_arr[i]
        e_dict[i] = []
        
        # find # of peaks
        peak = peak_arr[i]
        
        # loop through # of peaks
        for j in range(peak):
            e_dict[i].append(func(y,i,j,func_args[0]))
#             try:
#                 e_dict[i].append(func(y,i,j,func_args[0]))
#             except:
#                 print(i)
#                 print(j)
#                 break
   
            if func == timeToRest or func == atMax:
                e_dict[i][j] = e_dict[i][j] / .189
            elif func == tau or func == backTau:
                if e_dict[i][j] != None:
                    e_dict[i][j] = e_dict[i][j] / 18.9
    
    # create df
    f_dict = dict([ (k,pd.Series(v)) for k,v in e_dict.items() ])
    df = pd.DataFrame(f_dict)
    array = np.concatenate(list(f_dict.values()))
    return df, array

In [14]:
# Helper function to find start val based on percentage of max val
# y - Contractile data of one strain
# mid_val - max_val of peak
# thresh - percentage of max_val
# forward - if the val to find is in front of max or before
# RETURN - index of start_val
def findStartVal(y, mid_val, thresh, forward = True):
    # initial index of mid_val
    mid_idx = y.index(mid_val)
    # increment based on forward
    mult = [-1,1]
    increment = mult[forward]
    start_val = mid_val
    # loop until start_val drops below thresh
    while start_val > thresh * mid_val:
        # define new temp start_val
        start_val = y[mid_idx + increment]
        
        #determine if loop is finished
        if start_val < thresh * mid_val:
            break
            
        #continue with next idx
        mid_idx += increment
    
    
    return mid_idx

In [15]:
# This function helps find where the intial slope is above expected threshold
# y - Contractile data of one strain
# start - index of start value
# slope_thresh - threshold of slope to hit
# forward - if the val to find is in front of max or before
# RETURN - index of start if different and slope at start
def findInitSlope(y,start,slope_thresh, forward = True):
    # increment based on forward
    mult = [-1,1]
    increment = mult[forward]
    
    # array of 2 values since finding slopes between two points
    x_0 = np.arange(0, 2)
    
    #stop which is one after start
    stop = start + increment
    
    # array of 2 values between start and stop
    y_0 = y[min(start,stop):max(start,stop)+1]
    
    # absolute value of slope with poly fit at power of 1  
    slope = abs(np.polyfit(x_0,y_0,1)[0])
    
    # loop until slope is above thresh
    while slope < slope_thresh:
        # when can't go backwards anymore
        if stop == 0:
            break
            
        # increase/decrease start and stop one step
        start += increment
        stop = start + increment

        
        # find slope again
        y_0 = y[min(start,stop):max(start,stop)+1]
        slope = abs(np.polyfit(x_0,y_0,1)[0])
        
   
    return start,slope

In [16]:
# This function helps find where the end slope is below expected threshold
# y - Contractile data of one strain
# start - index of start value
# slope_thresh - threshold of slope to hit
# slope - slope found from intial slope
# forward - if the val to find is in front of max or before
# RETURN - index of end 
def findEndValSlope2(y,start,slope,slope_thresh, forward = True):
    # increment based on forward
    mult = [-1,1]
    increment = mult[forward]
    
    # array of 2 values since finding slopes between two points
    x_0 = np.arange(0, 2)
    
    # define what stop is
    stop = start + increment

    # loop until slope falls below thresh
    while slope > slope_thresh:
        
        # if can't go backwards anymore
        if stop == 0:
            break
        if stop == len(y) - 1:
            stop +=1
            break
        # shift one step
        start += increment
        stop += increment
        #calculate slope again
        y_0 = y[min(start,stop):max(start,stop)+1]
        slope = np.polyfit(x_0,y_0,1)[0]
        
        #determine if slope has changed
        if slope < slope_thresh:
            
            # go to previous step
            stop -= increment
            
    return stop

In [23]:
# This function helps find where the end slope is below expected threshold
# y - Contractile data of one strain
# start - index of start value
# slope_thresh - threshold of slope to hit
# slope - slope found from intial slope
# forward - if the val to find is in front of max or before
# RETURN - index of end 
def findEndValSlope(y,start,slope,slope_thresh, forward = True):
    # increment based on forward
    mult = [-1,1]
    increment = mult[forward]
    slope_thresh = slope_thresh * -1*increment
    slope = slope *-1*increment
    
    # array of 2 values since finding slopes between two points
    x_0 = np.arange(0, 2)
    
    # define what stop is
    stop = start + increment

    # loop until slope falls below thresh
    while slope < slope_thresh:
        
        # if can't go backwards anymore
        if stop == 0:
            break
        if stop == len(y) - 1:
            stop +=1
            break
        # shift one step
        start += increment
        stop += increment
        #calculate slope again
        y_0 = y[min(start,stop):max(start,stop)+1]
        slope = np.polyfit(x_0,y_0,1)[0]
        
        #determine if slope has changed
        if slope > slope_thresh:
            
            # go to previous step
            stop -= increment
            
    return stop

In [17]:
# This function determines the relaxation rate of the strain at each peak
# y - Contractile data of one strain
# col - strain number
# num - peak number
# df - dataframe of max vals used to find start val
# RETURN - slope of most linear region in peak

def tau(y, col, num, df, ):
    
    
    max_val = df.iloc[:,col][num]
    start = findStartVal(y,max_val, .9)
    
   
    #now find end with slope changes
 
    start, slope = findInitSlope(y,start,.05)
    stop = findEndValSlope(y,start,slope,.05)
    
    
        
    #end found
    
    x_f = np.arange(0,stop-start+1)
    y_f = y[start:stop+1]
       
    slope_f = abs(np.polyfit(x_f,y_f,1)[0])
    
    if slope_f > .28:
        slope_f = None
    
    return slope_f

In [1]:
def timeToRest(y, col, num, df):
    
    max_val = df.iloc[:,col][num]
    
    start = findStartVal(y,max_val, .9)

#     #now find end with slope changes
    t_start, slope = findInitSlope(y,start,.03)

    x_0 = np.arange(0,2)
    stop = t_start +1
    start_val = y[t_start]
    stop_val = y[stop]
   
    # loop until either stop val is below .1 or slope is less .01
    while stop_val > .1 or slope > .01:
        stop +=1
        t_start += 1
        
        
        if stop == len(y)-1:
            break
            
        stop_val = y[stop]
       
        if stop_val <= .1:
            stop -= 1
            break
            
        y_0 = y[min(t_start,stop):max(t_start,stop)+1]
        slope = abs(np.polyfit(x_0,y_0,1)[0])
        if slope <= .005:
            # go to previous step
            stop -= 1
            break
        
    

        
    #end found

    
    return stop - start

In [19]:
def backTau(y, col, num, df):
    #find where first goes below 90%
    max_val = df.iloc[:,col][num]
    end = findStartVal(y,max_val, .95, False)
        
    end, slope = findInitSlope(y,end,.1,False)

    start = findEndValSlope2(y, end,slope, .1, False)
#     #now find end with slope changes
#     temp_end = end
#     start = temp_end - 1
#     x_0 = np.arange(0, 2)
#     y_0 = y[start:temp_end+1]
#     slope = np.polyfit(x_0,y_0,1)[0] 
#     while slope < .1:
#         if start == 0:
#             break
#         end -= 1
#         temp_end = end
#         start = temp_end - 1
#         x_0 = np.arange(0, 2)
#         y_0 = y[start:temp_end+1]
#         slope = np.polyfit(x_0,y_0,1)[0] 
    
        
#     while slope > .1:
#         if start == 0:
#             break
#         temp_end -= 1
#         start = temp_end - 1
#         x_n = np.arange(0, 2)
#         y_n = y[start:temp_end+1]
#         slope = np.polyfit(x_n,y_n,1)[0] 
#         if slope < .1:
#             start += 1
        
#     #start found
    
    x_f = np.arange(0,end-start+1)
    y_f = y[start:end+1]
    slope_f = abs(np.polyfit(x_f,y_f,1)[0] )
    
    return slope_f

In [20]:
def atMax(y, col, num, df):
    #find where first goes below 95%
    max_val = df.iloc[:,col][num]
    max_idx = y.index(max_val)
    mid_val = max_val

    
    #find initial drop
    start_val = mid_val
    start_idx = max_idx
    above = start_val > .95 * max_val
    while above:
        start_val = y[start_idx-1]
        above = start_val > .95 * max_val
        if not above:
            break
        start_idx -= 1
    
    
    #find post drop
    end_val = mid_val
    end_idx = max_idx
    while above:
        end_val = y[end_idx+1]
        above = start_val > .95 * max_val
        if not above:
            break
        end_idx +=1
        
    return end_idx - start_idx

In [4]:
def add_annotations(array, df, tick_val, num):
    all_max = []
    [all_max.extend(v) for k,v in array.items()]
   
    all_max = [ ele for ele in all_max if ele is not None  ]
    max_val = max(all_max)
    min_val = min(all_max)
    

    # statistical annotation
    x1_vals = [1,1,1,0,0,2]
    x2_vals = [0,2,3,2,3,3]
    if num == 4:
        for i in range(6):
            x1 = x1_vals[i]
            x2 = x2_vals[i]
            p_value = df.iloc[i,2].round(3)
            if p_value < .05:
                p_value = "*"
            elif p_value <.01:
                p_value = "**"
            elif p_value <.001:
                p_value = "***"
            else:
                p_value = ""
            if i == 5:
                y, h, col = max_val + tick_val, tick_val, 'k'
            else:
                y, h, col = max_val + tick_val, (i+1)*tick_val, 'k'
            if i ==2 or i == 4:
                y = min_val - tick_val/2
                h = tick_val*(i/2)
                plt.plot([x1, x1, x2, x2], [y, y-h, y-h, y], lw=1.5, c=col)
                plt.text((x1+x2)*.5, y-h, p_value , ha='center', va='bottom', color=col)
            else:
                plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
            plt.text((x1+x2)*.5, y+h, p_value , ha='center', va='bottom', color=col)
    elif num == 3:
        x1_vals = [0,0,1]
        x2_vals = [1,2,2]
        for i in range(3):
            x1 = x1_vals[i]
            x2 = x2_vals[i]
            p_value = df.iloc[i,2].round(3)
            if p_value < .05:
                p_value = "*"
            elif p_value <.01:
                p_value = "**"
            elif p_value <.001:
                p_value = "***"
            else:
                p_value = ""
            if i == 5:
                y, h, col = max_val + tick_val, tick_val, 'k'
            else:
                y, h, col = max_val + tick_val, (i+1)*tick_val, 'k'
            if i ==1:
                y = min_val - tick_val/2
                h = tick_val*(i/2)
                plt.plot([x1, x1, x2, x2], [y, y-h, y-h, y], lw=1.5, c=col)
                plt.text((x1+x2)*.5, y-h, p_value , ha='center', va='bottom', color=col)
            else:
                plt.plot([x1, x1, x2, x2], [y, y+h, y+h, y], lw=1.5, c=col)
                plt.text((x1+x2)*.5, y+h, p_value , ha='center', va='bottom', color=col)
    else:
        pass

In [5]:
def statistics(array):
    df_val = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in array.items() ]))
    df_new = pd.melt(df_val.reset_index(),id_vars=['index'] )
    df_new['value'] = pd.to_numeric(df_new['value'])
    res.tukey_hsd(df=df_new, res_var='value', xfac_var='variable', anova_model='value ~ C(variable)')
    df = res.tukey_summary[["group1", "group2", "p-value"]]
    return df

In [17]:
def plot_box(array,num,val=4):
    f_dict = dict([ (k,pd.Series(v)) for k,v in array.items() ])
    df = pd.DataFrame(f_dict)
    g = sns.boxplot(data = df, palette = "summer", order = ordered)
    g.set_xlabel("Samples")
    df = statistics(array)
    num_dict = {0: 0, 1: 2, 2: -2, 3: 2, 4: -1 }
    y_label_dict = {
        0: "Percentage of Cells Contracting (%)",
        1: "Relaxation Rate of Cells Contracting (%cells/sec)",
        2: "Time of Cells Relaxing (ms)",
        3: "Rate of Contraction (%cells/sec)",
        4: "Time of Contraction (ms)"
    }
    
    all_max = []
    [all_max.extend(v) for k,v in array.items()]
    all_max = [ ele for ele in all_max if ele is not None  ]
    max_val = max(all_max)
    min_val = min(all_max)
    
 
    max_new = round_up(max_val, num_dict[num])
    
    tick_val = max_new/10

    
    add_annotations(array, df, tick_val,val)

    g.set_ylabel(y_label_dict[num])

    
    return g

In [1]:
def grouped_graph(array,num):
    day_list = ["D6", "D12", "D18"]
    df_empty = pd.DataFrame()
    for i in day_list:
        f_dict = dict([ (k,pd.Series(v)) for k,v in array[i].items() ])
        df = pd.DataFrame(f_dict)
        df = pd.DataFrame.from_dict(f_dict,orient='index').transpose()
        df_temp = pd.melt(df)
        df_temp["day"] = i
        df_empty = pd.concat([df_empty, df_temp])
        
    g = sns.boxplot(data = df_empty, x = "day", y = "value", hue = "variable",hue_order = ordered, palette = "summer")
    y_label_dict = {
        0: "Percentage of Cells Contracting (%)",
        1: "Relaxation Rate of Cells Contracting (%cells/sec)",
        2: "Time of Cells Relaxing (ms)",
        3: "Rate of Contraction (%cells/sec)",
        4: "Time of Contraction (ms)"
    }
    g.set_ylabel(y_label_dict[num])
    return g

In [7]:
def round_up(n, decimals=0):
    multiplier = 10**decimals
    return math.ceil(n * multiplier) / multiplier
def round_down(n, decimals=0):
    multiplier = 10**decimals
    return math.floor(n * multiplier) / multiplier
def myround(x, base=5):
    return base * round(x/base)

In [1]:
def array_to_df(data):
    max_length = max(len(item) for item in data.values())

    # Pad shorter arrays with np.nan
    for key in data.keys():
        current_length = len(data[key])
        if current_length < max_length:
            data[key] = np.pad(data[key], (0, max_length - current_length), constant_values=np.nan)

    # Now create the DataFrame
    df = pd.DataFrame(data)
    return df